# 多智能體系統簡介：基於 LangGraph 的架構導論

## 📘 前言
在人工智慧（AI）應用中，「智能體（Agent）」是一種能夠自動執行任務的系統，能根據環境輸入進行推理、決策並產生行動。傳統上，單一智能體可透過內建的工具（如資料檢索、API 操作或文字生成）來完成特定領域的工作。然而，即使使用像 **GPT-4** 這樣強大的模型，當任務涉及多個領域或需要同時使用多種工具時，單一智能體往往顯得力不從心。

為了克服這樣的限制，**多智能體系統（Multi-Agent System, MAS）** 應運而生。

---

## 🤖 多智能體網絡（Multi-Agent Network）
多智能體網絡是一種「**分而治之（divide-and-conquer）**」的設計思維。  
與其依賴單一大型模型處理所有任務，不如將任務拆解，交由多個專門化的智能體各司其職。

舉例來說：
- 一個智能體專門負責資料搜尋；
- 一個智能體專注於資料分析；
- 另一個智能體負責將結果轉換為自然語言報告。

這些智能體透過一個協調機制（Coordinator 或 Orchestrator）互相溝通，形成一個協作式網絡。這樣的架構不僅能提高效率，還能擴展系統的能力，使其能夠應對更複雜的工作流程。

---

## 🧠 為何採用多智能體架構？
| 挑戰 | 單智能體限制 | 多智能體解法 |
|------|---------------|---------------|
| 多領域專業知識 | 模型難以同時精通多領域 | 為每個領域建立專家型智能體 |
| 工具整合 | 工具過多導致決策困難 | 各智能體各自使用特定工具 |
| 任務複雜度 | 長鏈任務容易錯誤累積 | 透過分工與回饋提高準確率 |
| 可擴展性 | 模型難以動態調整 | 可彈性增減智能體節點 |

---

## 🏗️ LangGraph 中的多智能體架構範例
一個簡單的多智能體流程可能如下：

1. **任務路由代理（Router Agent）**  
   - 負責接收使用者的請求，並決定應交由哪個專家智能體處理。  

2. **專家智能體（Expert Agents）**  
   - 各自專精於特定領域（如數據分析、程式生成、文件撰寫）。  

3. **協調節點（Coordinator Node）**  
   - 收集並整合各專家智能體的輸出，生成最終結果。  

透過 LangGraph 的節點設計，可以輕鬆定義這樣的網絡結構，並使用圖的形式直觀地追蹤每個步驟。

---

## 🔍 小結
多智能體系統代表了人工智慧應用從「單一模型」邁向「協作式智能」的重要一步。  
在 **LangGraph** 的幫助下，開發者能以模組化與可視化的方式，構建能協同工作的智能體網絡，進而實現更高效、更具彈性的 AI 解決方案。

未來，隨著多智能體架構的成熟，我們將更接近一個「自組織式智能體社群（Agent Society）」的願景——讓 AI 不僅能「思考」，更能「協作」。

---


In [ ]:
import os

os.chdir("../../../")

## Researcher

In [ ]:
def make_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )

In [ ]:
from typing import Literal
from textwrap import dedent

from langgraph.graph import END
from langgraph.prebuilt import create_react_agent
from langgraph.types import Command
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI

from src.initialization import credential_init

credential_init()

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=6,
    disable_streaming=False
    # other params...
)

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)

# Research agent and node
research_agent = create_react_agent(
    model,
    tools=[tavily_tool],
    prompt=make_system_prompt(
        "You can only do research. You are working with a chart generator colleague."
    ),
)


In [ ]:
messages = [HumanMessage(content=dedent("""
                                         First, get the UK's GDP over the past 5 years, then make a line chart of it. 
                                         Once you make the chart, finish.
                                         """))]

researcher_response = research_agent.invoke({"messages": messages})

In [ ]:
researcher_response

In [ ]:
researcher_response["messages"][-1]

## Chart Agent

In [ ]:
# !pip install langchain-experimental matplotlib

In [ ]:
from typing import Annotated

from langchain_experimental.utilities import PythonREPL

"""
In frameworks such as LangChain, LlamaIndex, or OpenAI’s function calling examples, 
PythonREPL is often an execution tool or class that lets an AI agent safely run Python code.
"""
repl = PythonREPL()

@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

In [ ]:
chart_agent = create_react_agent(
    model,
    [python_repl_tool],
    prompt=make_system_prompt(
        "You can only generate charts. You are working with a researcher colleague."
    ),
)

In [ ]:
from langchain_core.messages import AIMessage

messages.append(researcher_response["messages"][-1])

# Reverse the role of ai and 
cls_map = {"ai": HumanMessage, "human": AIMessage}
    
messages_charter = [cls_map[msg.type](content=msg.content) for msg in messages]

charter_response = chart_agent.invoke({"messages": messages_charter}) 

In [ ]:
charter_response

## Langgraph Workflow

In [ ]:
from typing import Annotated, List, Sequence
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.types import Command


def get_next_node(last_message: BaseMessage, goto: str):
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return END
    return goto


class State(TypedDict):
    messages: Annotated[list, add_messages]


async def research_node(state: State):

    response = await research_agent.ainvoke({"messages": state["messages"]})

    last_message = response["messages"][-1]
    
    goto = get_next_node(last_message, 'chart')
    
    # return Command(update={"messages": response[-1]},
    #                goto=goto)
    return Command(update={"messages": state["messages"] + [last_message]},
                   goto=goto)


async def chart_node(state: State):
    
    # Reverse the role of ai and 
    cls_map = {"ai": HumanMessage, "human": AIMessage}
        
    messages_chart = [cls_map[msg.type](content=msg.content) for msg in state["messages"]]
    
    response = await chart_agent.ainvoke({"messages": messages_chart})

    last_message = HumanMessage(content=response["messages"][-1].content)
    
    goto = get_next_node(last_message, 'research')
    
    return Command(update={"messages": state["messages"] + [last_message]},
                   goto=goto)

In [ ]:
workflow = StateGraph(State)

workflow.add_node("research", research_node)
workflow.add_node("chart", chart_node)

workflow.add_edge(START, "research")

memory = InMemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
messages = [HumanMessage(content=dedent("""
                                         First, get the UK's GDP over the past 5 years, then make a line chart of it. 
                                         Once you make the chart, finish.
                                         """))]


In [ ]:
async for event in app.astream(
    {
        "messages": messages,
    },
    config,
):
    print(event)
    print("---")

In [ ]:
state = app.get_state(config)

In [ ]:
state.values